### Import Library yang dibutuhkan

In [ ]:
import mysql.connector # Import mysql.connector
from mysql.connector import Error # Import Error from mysql.connector
import re # Regular Expression
import numpy as np # Numerical Python (Numpy)
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory # Sastrawi Stemmer
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory # Sastrawi Stopword Remover
from nltk.tokenize import word_tokenize # Natural Language Toolkit (NLTK)
from sklearn.feature_extraction.text import TfidfVectorizer # Term Frequency-Inverse Document Frequency (TF-IDF)
from sklearn.metrics.pairwise import cosine_similarity # Cosine Similarity

### Koneksi Ke Database dan mengambil data dari database

In [ ]:
db_config = {
        "host": "mysql-students-my-student-projects.h.aivencloud.com",
        "port": "13993",
        "user": "galangarsandy",
        "password": "AVNS_1coj2um74BOgK_AGXaI",
        "database": "quran"
    }
class DatabaseConnection:
    def __init__(self):
        try:
            self.connection = mysql.connector.connect(**db_config)
            self.cursor = self.connection.cursor()
            print("Connected to the database\n")
        except Error as err:
            print(f"The error '{err}' occurred")
            
    def Select(self, sql):
        try:
            self.cursor.execute(sql)
            return self.cursor.fetchall()
        except Error as err:
            print(f"Select data ERROR: ", err)
    
    def __del__(self):
        self.connection.close()  
        
db = DatabaseConnection() # Membuat objek database connection
data_quran = db.Select("SELECT * FROM id_indonesian limit 50") # Mengambil data dari database

Connected to the database



### Buat Objek stemmer dan stopwords, dan inisiasi list data corpus

In [ ]:
stemmer = StemmerFactory().create_stemmer() # Membuat objek stemmer
stopwords = StopWordRemoverFactory().create_stop_word_remover() # Membuat objek stopwords

corpus = [] # List untuk menampung data quran yang sudah di pre-processing

### Fungsi untuk preprocessing data

In [ ]:
def pre_processing(text):
    pre_process = re.sub('[^a-zA-Z]', ' ', text.lower()) # Menghilaangkan karakter selain huruf a-z dan A-Z dan diubah menjadi spasi. Lalu diubah menjadi huruf kecil
    pre_process = re.sub(' +', ' ', pre_process) # Menghilangkan spasi yang berlebihan
    pre_process = stopwords.remove(pre_process) # Menghilangkan stopwords atau kata-kata yang tidak penting
    pre_process = stemmer.stem(pre_process) # Melakukan stemming atau mengubah kata-kata menjadi kata dasar
    return pre_process

### Preprocessing data_quran

In [ ]:
for (index, surat, ayat, text) in data_quran:
    pre_process_data = pre_processing(text)
    corpus.append((index, surat, ayat, pre_process_data)) # Menambahkan data ke dalam list corpus

### Tokenisasi

In [ ]:
doc_vocabulary = set()
tokenized_documents = []

for (index, surat, ayat, text) in corpus:
    tokens = word_tokenize(text) # Tokenisasi kata atau memecah kalimat menjadi per kata
    tokens = [word.lower() for word in tokens if word.isalpha()] # Menghilangkan kata yang bukan alphabet, Misal tokens = ["Hello", "world!", "123", "Python3"] menjadi ["hello", "world", "python"]
    tokenized_documents.append(" ".join(tokens)) # Menambahkan kata-kata yang sudah di tokenisasi ke dalam list tokenized_documents
    doc_vocabulary.update(tokens) # Menambahkan kata-kata yang unik ke dalam set vocabulary, jadi tidak ada kata yang duplikat

### Pembobotan Kata dengan TF-IDF

In [ ]:
vectorizer = TfidfVectorizer(vocabulary=doc_vocabulary) # Membuat objek vectorizer dengan parameter vocabulary yang berisi kata-kata unik
tfidf_matrix = vectorizer.fit_transform(tokenized_documents) # Mengubah kata-kata menjadi vektor dengan metode TF-IDF, atau menghitung bobot dari masing masing kata

### Coba untuk melakukan pencarian, dengan menggunakan cosine_similarity untuk menghitung kemiripan antara query dengan dokumen yang ada di database

In [ ]:
query = input("Masukkan query: ")
query_tokens = word_tokenize(query) # Tokenisasi kata atau memecah kalimat menjadi per kata
query_tokens = [word.lower() for word in query_tokens if word.isalpha()]
query_vector = vectorizer.transform([" ".join(query_tokens)]) # Mengubah kata-kata menjadi vektor

cosine_similarities = cosine_similarity(tfidf_matrix, query_vector) # Menghitung cosine similarity antara tfidf_matrix dan query_vector
most_similar_document_index = np.argmax(cosine_similarities) # Mengambil index yang memiliki nilai cosine similarity paling besar
most_similar_document = data_quran[most_similar_document_index] # Mengambil data quran yang memiliki nilai cosine similarity paling besar

print("\nDokumen paling mirip dengan queri:")
print(most_similar_document)


Dokumen paling mirip dengan queri:
(10, 2, 3, '(yaitu) mereka yang beriman kepada yang ghaib, yang mendirikan shalat, dan menafkahkan sebahagian rezeki yang Kami anugerahkan kepada mereka.')


## Jawaban Soal Praktikum No 1

1. tokenized_documents:
Ini adalah daftar yang berisi dokumen-dokumen yang telah diproses dan di-tokenisasi. Setiap dokumen diwakili sebagai string yang berisi kata-kata yang dipisahkan oleh spasi.
2. doc_vocabulary:
Ini adalah himpunan (set) yang berisi semua kata unik yang ditemukan dalam korpus dokumen setelah pemrosesan.
3. cosine_similarities: Ini adalah array yang berisi nilai kesamaan kosinus antara query yang diberikan dan setiap dokumen dalam korpus. Nilai ini menunjukkan seberapa mirip query dengan setiap dokumen.

## Jawaban Soal Praktikum No 2

Tinggal gunakan fungsi pre-processing data yang sudah dibuat
query = pre_processing(query)
```
query = pre_processing(query)
```

In [ ]:
input_query = input("Masukkan query: ")
query = pre_processing(input_query) # Gunakan Pre-processing text untuk input_query
query_tokens = word_tokenize(query) 
query_tokens = [word.lower() for word in query_tokens if word.isalpha()]
query_vector = vectorizer.transform([" ".join(query_tokens)])

cosine_similarities = cosine_similarity(tfidf_matrix, query_vector) 
most_similar_document_index = np.argmax(cosine_similarities) 
most_similar_document = data_quran[most_similar_document_index]

print(f"\nDokumen yang mirip dengan query '{input_query}':")
print(most_similar_document)


Dokumen yang mirip dengan query 'Allah, maha pengasih & penyayang':
(3, 1, 3, 'Maha Pemurah lagi Maha Penyayang.')


## Jawaban Soal Praktikum No 3

In [ ]:
input_query = input("Masukkan query: ")
query = pre_processing(input_query)
query_tokens = word_tokenize(query) 
query_tokens = [word.lower() for word in query_tokens if word.isalpha()]
query_vector = vectorizer.transform([" ".join(query_tokens)])
cosine_similarities = cosine_similarity(tfidf_matrix, query_vector) 

# Jawaban No 3
# cosine_similarities di flatten agar menjadi 1 dimensional array
cosine_results = cosine_similarities.flatten()
# Mengambil index yang memiliki nilai cosine similarity lebih dari 0
index_sorted = np.where(cosine_results > 0)[0]
# Mengurutkan index berdasarkan nilai cosine similarity dari yang terbesar
index_sorted = index_sorted[np.argsort(cosine_results[index_sorted])[::-1]]

print(f"\nDokumen-dokumen yang mirip dengan query '{input_query}':")
for index in index_sorted:
    print(data_quran[index])


Dokumen-dokumen yang mirip dengan query 'maha sayang':
(3, 1, 3, 'Maha Pemurah lagi Maha Penyayang.')
(1, 1, 1, 'Dengan menyebut nama Allah Yang Maha Pemurah lagi Maha Penyayang.')
(44, 2, 37, 'Kemudian Adam menerima beberapa kalimat dari Tuhannya, maka Allah menerima taubatnya. Sesungguhnya Allah Maha Penerima taubat lagi Maha Penyayang.')
(39, 2, 32, 'Mereka menjawab: "Maha Suci Engkau, tidak ada yang kami ketahui selain dari apa yang telah Engkau ajarkan kepada kami; sesungguhnya Engkaulah Yang Maha Mengetahui lagi Maha Bijaksana".')
(36, 2, 29, 'Dialah Allah, yang menjadikan segala yang ada di bumi untuk kamu dan Dia berkehendak (menciptakan) langit, lalu dijadikan-Nya tujuh langit. Dan Dia Maha Mengetahui segala sesuatu.')
